In [1]:
// id : ex_4.1
sealed trait Option[+A] {
    def map[B](f: A => B) : Option[B] = {
        this match {
            case Some(a) => Some(f(a))
            case None => None
        }
    }
    def flatMap[B](f: A => Option[B]) : Option[B] = {
        this match {
            case Some(a) => f(a)
            case None => None
        }
    }
    def getOrElse[B >: A](default: => B) : B = {
        this match {
            case Some(a) => a
            case None => default
        }
    }
    def orElse[B >: A](ob: => Option[B]) : Option[B] = {
        this match {
            case Some(a) => Some(a)
            case None => ob
        }
    }
    def filter(f: A => Boolean) : Option[A] = {
        this match {
            case Some(a) => if( f(a) ) { this } else { None }
            case None => None
        }
    }
}
case class Some[+A](get: A) extends Option[A]
case object None extends Option[Nothing]

None

In [2]:
// id : try ex_4.1
println(Some(3).map(_ + 1))
val NoneInt : Option[Int] = None
println(NoneInt.map(_ + 1))
println(Some(3).flatMap((e : Int) => Some("\"" + e.toString + "\"")))
println(Some(3).flatMap((e : Int) => None))
println(NoneInt.flatMap((e : Int) => Some("\"" + e.toString + "\"")))
println(Some(3).getOrElse(6))
println(None.getOrElse(6))
println(Some(3).orElse(Some(4)))
println(None.orElse(Some(4)))
println(None.orElse(None))
println(Some(3).filter(_==2))
println(Some(3).filter(_==3))
println(None.filter(_==3))

Some(4)
None
Some("3")
None
None
3
6
Some(3)
Some(4)
None
None
Some(3)
None


null

In [3]:
// id : ex_4.2
def variance(xs: Seq[Double]) : Option[Double] = {
    if (xs.isEmpty) {
        None
    } else {
        val size = xs.size
        val mean = xs.sum / size
        val sigma = xs.map((e: Double) => (math.pow(e-mean,2))).sum / size
        Some(sigma)
    }
}

variance: (xs: Seq[Double])Option[Double]


In [4]:
// id : try ex_4.2
println(variance(Seq(600,470,170,430,300)))
// http://www.mathsisfun.com/data/standard-deviation.html :)
println(variance(Seq(1,3)))
println(variance(Seq(1)))
println(variance(Seq()))

Some(21704.0)
Some(1.0)
Some(0.0)
None


null

In [5]:
def Try[A](a : => A): Option[A] = try Some(a) catch { case e: Exception => None }

Try: [A](a: => A)Option[A]


In [6]:
// id : ex_4.3

def map2[A, B, C](a: Option[A], b: Option[B])(f: (A, B) => C) : Option[C] = {
    a match {
        case None => None
        case Some(ak) => {
            b match {
                case None => None
                case Some(bk) => Try(f(ak, bk))
            }
        }
    }
}

map2: [A, B, C](a: Option[A], b: Option[B])(f: (A, B) => C)Option[C]


In [7]:
// id : try ex_4.3

def maybeFails(x : String, y : String) : Int = (x + y).toInt
println(map2(Some("12"), Some("13"))(maybeFails))
println(map2(Some("12"), Some("ab"))(maybeFails))

Some(1213)
None


null

In [12]:
// id : ex_4.4

def sequence[A](as : List[Option[A]]) : Option[List[A]] = {
    as match {
        case Nil => Some(Nil)
        case x::xs => x match {
            case None => None
            case Some(b) => map2(x, sequence(xs))(_ :: _)
        }
    }
}

sequence: [A](as: List[Option[A]])Option[List[A]]


In [14]:
// id : try ex_4.4

println(sequence(List(Some("A"), Some("B"), Some("C"))))
println(sequence(List(Some("A"), None, Some("C"))))

Some(List(A, B, C))
None


null